In [ ]:
## 1

In [7]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.preprocessing import MinMaxScaler

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'mckayla-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to read from s3 bucket
file_key = 'College.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

##Reading the csv file
college = pd.read_csv(file_content_stream)
college.head()

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
Adrian College,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
Agnes Scott College,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
Alaska Pacific University,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


In [9]:
## Changing Private to a numerical variable
college['Private'] = np.where(college['Private'] == 'Yes',0, 1)
college.head()

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
Abilene Christian University,0,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
Adelphi University,0,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
Adrian College,0,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
Agnes Scott College,0,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
Alaska Pacific University,0,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


In [10]:
## Defining the input and target variables
X = college[['Private','F.Undergrad','P.Undergrad','Outstate','Room.Board', 'Books', 'Personal', 'S.F.Ratio', 'Grad.Rate']]
Y = college['Apps']

## SPlitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [11]:
## Min-Max transformation
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [12]:
##Building the linear model
lm_md = LinearRegression().fit(X_train, Y_train)

##Prediciting the test
lm_md_pred = lm_md.predict(X_test)

##Computing the model MSE
mse1 = np.mean((Y_test - lm_md_pred)** 2)
print('The MSE of model 1 is', mse1)

The MSE of model 1 is 2698955.3389719524


In [13]:
## Ridge Regression
ridge_cv = RidgeCV(np.linspace(0.001, 100, num = 100), cv = 5).fit(X_train, Y_train)

## Extracting the best lambda
cv_lambda = ridge_cv.alpha_

## Building the ridge model
ridge_md = Ridge(alpha = cv_lambda).fit(X_train, Y_train)

## Predicting on test
ridge_pred = ridge_md.predict(X_test)

## Computing the mse of the ridge regression model
mse2 = np.mean(np.power(Y_test - ridge_pred, 2))
print('The mse of the ridge model is', mse2)

The mse of the ridge model is 2758464.812703545


In [14]:
##Estimating Lambda for lasso
lasso_cv = LassoCV(alphas =[0.001, 100], normalize = True, cv = 5).fit(X_train, Y_train)

## Extracting the best lambda
cv_lambda = lasso_cv.alpha_

##Building Lasso Model
lasso_md = Lasso(alpha = cv_lambda, normalize = True).fit(X_train, Y_train)

## Predicting on test
lasso_pred = lasso_md.predict(X_test)

#Computing the mse of the lasso regression model
mse3 = np.mean(np.power(Y_test - lasso_pred, 2))
print('The mse of the lasso model is', mse3)

The mse of the lasso model is 2698949.4026707723


/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing

In [ ]:
## Based on the data provided the model to use to predict the number of applications that a university received would be the Lasso Model.

In [ ]:
## 2

In [26]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.metrics import recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold, train_test_split

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'mckayla-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to read from s3 bucket
file_key = 'churn-bigml-80.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

telecom_train = pd.read_csv(file_content_stream)
telecom_train.head()

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [21]:
## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'mckayla-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to read from s3 bucket
file_key = 'churn-bigml-20.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

telecom_test = pd.read_csv(file_content_stream)
telecom_test.head()

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,LA,117,408,No,No,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,False
1,IN,65,415,No,No,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.40,12.7,6,3.43,4,True
2,NY,161,415,No,No,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,True
3,SC,111,415,No,No,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,False
4,HI,49,510,No,No,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.00,1,False


In [23]:
## Changing Churn to a numerical variable
telecom_train['Churn'] = np.where(telecom_train['Churn'] == False, 0, 1)
telecom_test['Churn'] = np.where(telecom_test['Churn'] == False, 0, 1)
telecom_train.head()
telecom_test.head()

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,LA,117,408,No,No,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,0
1,IN,65,415,No,No,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.40,12.7,6,3.43,4,1
2,NY,161,415,No,No,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,1
3,SC,111,415,No,No,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,0
4,HI,49,510,No,No,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.00,1,0


In [30]:
## Changing International_plan to a numerical variable
telecom_train['International_plan'] = np.where(telecom_train['International_plan'] == 'No', 0, 1)
telecom_test['International_plan'] = np.where(telecom_test['International_plan'] == 'No', 0, 1)
telecom_train.head()
telecom_test.head()

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,LA,117,408,1,1,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,0
1,IN,65,415,1,1,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.40,12.7,6,3.43,4,1
2,NY,161,415,1,1,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,1
3,SC,111,415,1,1,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,0
4,HI,49,510,1,1,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.00,1,0


In [31]:
## Changing Voice_mail_plan to a numerical variable
telecom_train['Voice_mail_plan'] = np.where(telecom_train['Voice_mail_plan'] == 'No', 0, 1)
telecom_test['Voice_mail_plan'] = np.where(telecom_test['Voice_mail_plan'] == 'No', 0, 1)
telecom_train.head()
telecom_test.head()

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,LA,117,408,1,1,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,0
1,IN,65,415,1,1,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.40,12.7,6,3.43,4,1
2,NY,161,415,1,1,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,1
3,SC,111,415,1,1,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,0
4,HI,49,510,1,1,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.00,1,0


In [33]:
## Creating a new variable - total_charge
telecom_train['total_charge'] = telecom_train['Total_day_charge'] + telecom_train['Total_eve_charge'] + telecom_train['Total_night_charge'] + telecom_train['Total_intl_charge']
telecom_test['total_charge'] = telecom_test['Total_day_charge'] + telecom_test['Total_eve_charge'] + telecom_test['Total_night_charge'] + telecom_test['Total_intl_charge']

In [35]:
## Keeping these variables
telecom_train = telecom_train[['Account_length', 'International_plan', 'Voice_mail_plan', 'total_charge', 'Customer_service_calls', 'Churn']]
telecom_train.head()

,Account_length,International_plan,Voice_mail_plan,total_charge,Customer_service_calls,Churn
0,128,0,1,75.56,1,False
1,107,0,1,59.24,1,False
2,137,0,0,62.29,0,False
3,84,1,0,66.80,2,False
4,75,1,0,52.09,3,False


In [36]:
telecom_test = telecom_test[['Account_length', 'International_plan', 'Voice_mail_plan', 'total_charge', 'Customer_service_calls', 'Churn']]
telecom_test.head()

,Account_length,International_plan,Voice_mail_plan,total_charge,Customer_service_calls,Churn
0,117,1,1,73.32,1,0
1,65,1,1,54.20,4,1
2,161,1,1,92.29,4,1
3,111,1,1,41.05,2,0
4,49,1,1,49.60,1,0


In [37]:
## Defining the input and target variables
X = telecom_train[['Account_length', 'International_plan', 'Voice_mail_plan', 'total_charge', 'Customer_service_calls']]
Y = telecom_train['Churn']

## Splitting the data into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify = Y)

In [38]:
## Min-Max transformation
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [39]:
## Defining list to store estimated coefficients at each split
coef = list()

for i in range(0, 1000):
    
    ## Splitting the data into train and test
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2)

    ## Lasso cross-validation to estimate optimal lambda
    lasso_cv = LassoCV(normalize = True, cv = 5).fit(X_train, Y_train)

    ## Building Lasso Regression Model with optimal lambda
    lasso_md = Lasso(alpha = lasso_cv.alpha_, normalize = True).fit(X_train, Y_train)
    
    ## Storing the estimated coefficients
    coef.append(lasso_md.coef_)
    
## Putting the list as a dataframe
coef_df = pd.DataFrame(coef)

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing

In [40]:
coef_df.head()

,0,1,2,3,4
0,0.000055,0.299217,-0.065826,0.007200,0.056153
1,0.000166,0.335453,-0.083304,0.007503,0.059217
2,0.000034,0.300037,-0.068837,0.006321,0.060103
3,0.000037,0.322367,-0.080141,0.006797,0.052642
4,0.000095,0.325685,-0.087602,0.006880,0.056002


In [41]:
sum(coef_df[0] == 0.0)

254

In [58]:
from sklearn.metrics import recall_score

## Dropping Account_length
X = telecom_train[['International_plan', 'Voice_mail_plan', 'total_charge', 'Customer_service_calls']]
Y = telecom_train['Churn']

## Defining lists
md1_results = list()
md2_results = list()
md3_results = list()
md4_results = list()
md5_results = list()

for i in range(0,100):
    
    kfold = StratifiedKFold(n_splits = 5, shuffle = True)
    
    ## Defining lists to store
    md1_foldresults = list()
    md2_foldresults = list()
    md3_foldresults = list()
    md4_foldresults = list()
    md5_foldresults = list()
    
    for train_ix, test_ix in kfold.split(X, Y):
        
        ## Splitting data
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
        
        ## Transformation
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.fit_transform(X_test)

In [63]:
## Building the logistic model
logit1_results = LogisticRegression(solver = 'liblinear', penalty = 'l1').fit(X_train, Y_train)

##Predicting the test dataset
logit1_pred = logit1_results.predict_proba(X_test)[:,1]
logit1_pred

array([0.02677626, 0.12683687, 0.31490635, 0.02055363, 0.01990496,
       0.16211594, 0.01557501, 0.04662431, 0.0516244 , 0.29551808,
       0.09038779, 0.08979374, 0.05690789, 0.04588369, 0.20434772,
       0.07509682, 0.03441206, 0.0352238 , 0.08215779, 0.26742048,
       0.34239105, 0.20681337, 0.16159179, 0.16592484, 0.07453234,
       0.0399056 , 0.1170312 , 0.17663689, 0.07976747, 0.09530547,
       0.04664582, 0.08440745, 0.01714318, 0.06699042, 0.01360855,
       0.17690053, 0.01798398, 0.15069429, 0.46719404, 0.29669248,
       0.0801362 , 0.20800467, 0.03503957, 0.0687438 , 0.11351488,
       0.02174738, 0.02089069, 0.19790547, 0.04141534, 0.27431405,
       0.04384751, 0.0246924 , 0.03047022, 0.03014901, 0.13502373,
       0.06392826, 0.02518734, 0.01850313, 0.66004623, 0.32671733,
       0.26838357, 0.41179574, 0.12513923, 0.0954355 , 0.13985547,
       0.07267938, 0.0596179 , 0.0200054 , 0.04266039, 0.04167987,
       0.09029841, 0.02501439, 0.18033823, 0.0227925 , 0.06269

In [64]:
## Changing lielihoods to labels
logit1_labels = np.where(logit1_pred < 0.10, 0 , 1)
logit1_labels
recall_score(Y_test, logit1_labels)

0.782051282051282

In [65]:
## Building the logistic model
logit2_results = LogisticRegression(solver = 'liblinear', penalty = 'l2').fit(X_train, Y_train)

##Predicting the test dataset
logit2_pred = logit2_results.predict_proba(X_test)[:,1]
logit2_pred

array([0.03438668, 0.17507449, 0.29712273, 0.03379085, 0.03193821,
       0.16870217, 0.02210056, 0.06244941, 0.07171907, 0.28113838,
       0.10256647, 0.10205963, 0.07250179, 0.06366606, 0.20114157,
       0.07508998, 0.04974194, 0.05061865, 0.09547156, 0.25387305,
       0.30838967, 0.20299525, 0.1591519 , 0.17168967, 0.08874446,
       0.04799223, 0.12458928, 0.18001945, 0.09059199, 0.09284886,
       0.06054439, 0.10981495, 0.02854315, 0.07324474, 0.02061867,
       0.18022319, 0.02545532, 0.15090072, 0.45486021, 0.33816695,
       0.08649081, 0.20388953, 0.05042012, 0.08100624, 0.10935672,
       0.0293855 , 0.0285066 , 0.18596186, 0.05537516, 0.25258764,
       0.04992376, 0.03755607, 0.04398152, 0.03883848, 0.1249364 ,
       0.07910233, 0.03283352, 0.02600868, 0.59136933, 0.27659547,
       0.30055786, 0.42089182, 0.1349063 , 0.11006951, 0.15093385,
       0.08708513, 0.07278739, 0.02670753, 0.0584292 , 0.05564036,
       0.10560158, 0.03792348, 0.17793942, 0.03651925, 0.06545

In [66]:
## Changing lielihoods to labels
logit2_labels = np.where(logit2_pred < 0.10, 0 , 1)
logit2_labels
recall_score(Y_test, logit2_labels)

0.8205128205128205

In [68]:
## Building the logistic model
logit3_results = LogisticRegression(solver = 'saga', penalty = 'l1').fit(X_train, Y_train)

##Predicting the test dataset
logit3_pred = logit1_results.predict_proba(X_test)[:,1]
logit3_pred

array([0.02677626, 0.12683687, 0.31490635, 0.02055363, 0.01990496,
       0.16211594, 0.01557501, 0.04662431, 0.0516244 , 0.29551808,
       0.09038779, 0.08979374, 0.05690789, 0.04588369, 0.20434772,
       0.07509682, 0.03441206, 0.0352238 , 0.08215779, 0.26742048,
       0.34239105, 0.20681337, 0.16159179, 0.16592484, 0.07453234,
       0.0399056 , 0.1170312 , 0.17663689, 0.07976747, 0.09530547,
       0.04664582, 0.08440745, 0.01714318, 0.06699042, 0.01360855,
       0.17690053, 0.01798398, 0.15069429, 0.46719404, 0.29669248,
       0.0801362 , 0.20800467, 0.03503957, 0.0687438 , 0.11351488,
       0.02174738, 0.02089069, 0.19790547, 0.04141534, 0.27431405,
       0.04384751, 0.0246924 , 0.03047022, 0.03014901, 0.13502373,
       0.06392826, 0.02518734, 0.01850313, 0.66004623, 0.32671733,
       0.26838357, 0.41179574, 0.12513923, 0.0954355 , 0.13985547,
       0.07267938, 0.0596179 , 0.0200054 , 0.04266039, 0.04167987,
       0.09029841, 0.02501439, 0.18033823, 0.0227925 , 0.06269

In [69]:
## Changing lielihoods to labels
logit3_labels = np.where(logit3_pred < 0.10, 0 , 1)
logit3_labels
recall_score(Y_test, logit3_labels)

0.782051282051282

In [70]:
## Building the logistic model
logit4_results = LogisticRegression(solver = 'saga', penalty = 'l2').fit(X_train, Y_train)

##Predicting the test dataset
logit4_pred = logit4_results.predict_proba(X_test)[:,1]
logit4_pred

array([0.03136417, 0.15320448, 0.31103948, 0.02715382, 0.02636993,
       0.16187329, 0.01959198, 0.05503282, 0.06026766, 0.27520673,
       0.09736058, 0.09680836, 0.06534297, 0.05436329, 0.19811096,
       0.07659965, 0.04235842, 0.04321845, 0.08966582, 0.25145672,
       0.31385157, 0.2002042 , 0.16097303, 0.16517637, 0.08244414,
       0.04436541, 0.12169356, 0.17442504, 0.08728094, 0.09436582,
       0.05496913, 0.09232914, 0.02317801, 0.06958613, 0.01745934,
       0.17465194, 0.02222516, 0.15149449, 0.46335579, 0.31440457,
       0.08141145, 0.20121482, 0.0430235 , 0.07677889, 0.11005888,
       0.02619766, 0.02530219, 0.19212023, 0.04961423, 0.25689689,
       0.04805802, 0.03176248, 0.03807966, 0.03480975, 0.12809051,
       0.07223122, 0.02974725, 0.0227794 , 0.61391494, 0.29978503,
       0.28815193, 0.4155897 , 0.12913906, 0.10218501, 0.1423939 ,
       0.08067461, 0.06791088, 0.02433248, 0.05097662, 0.04988723,
       0.09742215, 0.03211959, 0.17736739, 0.02968907, 0.06549

In [71]:
## Changing likelihoods to labels
logit4_labels = np.where(logit4_pred < 0.10, 0 , 1)
logit4_labels
recall_score(Y_test, logit4_labels)

0.7948717948717948